In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
user_rating = pd.read_csv('data\movies.csv')
tmdb = pd.read_csv('data\\tmdb_movies_data.csv')
user_rating["title"]= user_rating["title"].apply(lambda x: re.sub(r"\([^)]*\)", "", x))
user_rating["title"] = user_rating["title"].apply(lambda x: x.strip())

In [4]:
#match the movie titles in both the dataframes
user_rating_movies = user_rating["title"].unique()
tmdb_movies = tmdb["original_title"].unique()
common = np.intersect1d(user_rating_movies, tmdb_movies)
user_rating = user_rating[user_rating["title"].isin(common)]
existing_movie_ids = user_rating["movieId"].unique()

In [15]:
#filter the ratings dataframe to only include the movies that are present in the tmdb dataframe and select the users who have rated significant amount of movies
ratings = pd.read_csv('data\\ratings.csv')
ratings = ratings[ratings["movieId"].isin(existing_movie_ids)]
selected_users = ratings.groupby("userId").count().sort_values("movieId", ascending=False)
selected_users = selected_users[selected_users["movieId"]*0.2 > 20].index

In [16]:
#match the movie names and make ratings binary
ratings = ratings[ratings["userId"].isin(selected_users)]
ratings["movieTitle"] = ratings["movieId"].apply(lambda x: user_rating[user_rating["movieId"]==x]["title"].values[0])
ratings["rating"] = ratings["rating"].apply(lambda x: int(x))
ratings["rating"] = ratings["rating"].apply(lambda x: 1 if x>=3 else 0)
ratings.to_csv("data\\ratings_selected.csv", index=False)